In [1]:
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

import pandas as pd
import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#  Dados de desempenho do time masculino de futebol do Cruzeiro 2014 - 2022
navegador = webdriver.Chrome()

In [3]:
a = 0  
url = "https://fbref.com/pt/equipes/03ff5eeb/Cruzeiro-Estatisticas"
resumo = []

In [4]:
while a in range(0,9):
    
    navegador.get(url)
    navegador.find_element("xpath", "/html/body/div[2]/div[3]/div[1]/div[2]/div/a/div").click()
    
    url = navegador.current_url
    headers ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}

    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html,'html.parser')
    lista = soup.find('table').findNext('table')

#  Ações e suas respectivas quantidades
    registro = lista.findAll('th', class_ = 'left') #  os registros da tabela
    qtd_1 = range(int(len(registro)))

#  Informações da primeira linha que não entram no loop
    data =  lista.find('th', class_='left').getText()
    horario = lista.find('td').contents[0]

#  Para cada linha (i) em registro, obtenha os seguintes conteúdos:
    for i in registro:
        cruzeiro ={}
        camp = horario.findNext('td').contents[0]
        rodada = camp.findNext('td').contents[0]
        dia = rodada.findNext('td').contents[0]
        local = dia.findNext('td').contents[0]
        resultado = local.findNext('td').contents[0]
        gp = resultado.findNext('td').contents[0]
        gc = gp.findNext('td').contents[0]
        oponente = gc.findNext('td').contents[0]

        
#  Armazene as informações em cruzeiro da seguinte forma:
        cruzeiro['Data'] = data
        cruzeiro['Horário'] = horario
        cruzeiro['Campeonato'] = camp
        cruzeiro['Rodada'] = rodada
        cruzeiro['Dia'] = dia
        cruzeiro['Local']= local
        cruzeiro['Resultado'] = resultado
        cruzeiro['GP'] = gp
        cruzeiro['GC'] = gc
        cruzeiro['Oponente'] = oponente

        
#  Armazene cruzeiro na lista vazia criada anteriormente
        resumo.append(cruzeiro)
    
#  A última linha deve dar erro, pois não tem próxima etapa. Por isso, é necessária uma exceção:
        try:
            data = i
            horario = data.findNext('td').contents[0]
        except:
            pass
    
    a += 1

In [5]:
#  Cria data frame com dados extraídos anteriormente
df = pd.DataFrame(resumo)
 
display(df)

,Data,Horário,Campeonato,Rodada,Dia,Local,Resultado,GP,GC,Oponente
0,2022-04-08,[21:30],[Série B],[Rodada da semana 1],sex,Visitante,D,0,2,[Bahia]
1,[[2022-04-08]],[21:30],[Série B],[Rodada da semana 1],sex,Visitante,D,0,2,[Bahia]
2,[[2022-04-12]],[21:30],[Série B],[Rodada da semana 2],ter,Em casa,V,1,0,[Brusque Futebol Clube]
3,[[2022-04-23]],[19:00],[Série B],[Rodada da semana 3],sáb,Visitante,E,1,1,[Tombense Futebol Clube]
4,[[2022-04-26]],[21:30],[Série B],[Rodada da semana 4],ter,Em casa,V,1,0,[Londrina]
...,...,...,...,...,...,...,...,...,...,...
377,[[2014-11-09]],[19:30],[Série A],[Rodada da semana 33],dom,Em casa,V,3,1,[Criciúma]
378,[[2014-11-16]],[17:00],[Série A],[Rodada da semana 34],dom,Visitante,V,1,0,[Santos]
379,[[2014-11-20]],[21:50],[Série A],[Rodada da semana 35],qui,Visitante,V,2,1,[Grêmio]
380,[[2014-11-23]],[17:00],[Série A],[Rodada da semana 36],dom,Em casa,V,2,1,[Goiás]
